In [1]:
from utils import *
from originalWordLists import guesses, answers
from tqdm.auto import tqdm
from valuations import *
import numpy as np

c:\Users\Michael\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
G = sorted(guesses + answers)
S = sorted(answers)

In [3]:
len(G)

12972

In [26]:
pG = list(G)
pS = list(S)
pG = filterPossible('salet', check('salet','wound') , pG)
pS = filterPossible('salet', check('salet','wound') , pS)
sortWords(pG, pS, [mostParts, firstValid, information], n=5)

NameError: name 'sortWords' is not defined

In [ ]:
# def f(A, first = False):
#     if len(A) == 1: return 1
#     if len(A) == 2: return 1.5
#     bestScore = 10000
#     for g in tqdm(sorted(G, key = lambda x: (x not in A, maxSizeSplit(x,A)))[:20], disable = not first):
#         t = 1
#         splits = getSplits(g, A, useWords = True)
#         if len(splits) == 1: continue
#         for res,split in splits.items():
#             t += len(split)/len(A) * f(split)
#         bestScore = min(bestScore, t)
#         if len(A) >= 7:
#             print(A,g,'-->',t)
#         if len(A) == 3 and (g in A and abs(t - 5/3) < 0.01):
# #             print("***exited early***")
#             return t
#         if len(A) == 4 and (g in A and abs(t - (1+2*3)/4) < 0.01):
#             print("***exited early***")
#             return t
#         if (g not in A) and abs(t - 2) < 0.01:
#             if len(A) >= 7: print("***exited early***")
#             return t
#     return bestScore

In [ ]:
# f(C, first = True)
# f(['rover', 'rower', 'wooer'], first = True)

In [ ]:
for k,v in sorted(getSplits('trace',S).items(), key = lambda x: x[1]):
    print(k,v)

In [ ]:
import json

In [ ]:
tree = json.load(open('combinedVals/mostParts,firstValid,maxSizeSplit,information1.json'))
# tree = tree['splits']['00100']
# tree = tree['splits']['00020']
# tree = tree['splits']['02000'] a
tree

In [ ]:
def g(T, words):
#     r = [1]
#     if 'splits' in T:
#         for k in T['splits']:
#             r += [1 + x for x in g(T['splits'][k])]
#     return r
    r = []
    for w in words:
        s = 1
        curr = T
        res = check(curr['guess'],w)
        while res != '22222':
            s += 1
            curr = curr['splits'][res]
            res = check(curr['guess'],w)
            
        r.append(s)
#         print(w,s)
    return r
            
        

In [ ]:
np.mean(g(tree,C)), len(g(tree,C)), max(g(tree,C))

In [ ]:
g(tree,C), tree
# ['boxer', 'homer', 'hover', 'joker', 'mover', 'roger', 'rover']
# ['berry',
#  'demur',
#  'derby',
#  'every',
#  'femur',
#  'ferry',
#  'jerky',
#  'leery',
#  'lemur',
#  'merry',
#  'query',
#  'reedy']

In [ ]:
%%time
get('tares', S)

In [ ]:
from strategyTree import TreeGenerator

In [ ]:
x = g(t)
np.mean(x), max(x)

In [ ]:
from itertools import permutations

In [ ]:
allVals = [firstValid, maxSizeSplit, mostParts, information]
fns = sum([
    list(permutations(allVals, i)) for i in range(1,len(allVals)+1)
], [])
ans = []
for f in fns:
    t = TreeGenerator(vs = f)
    x = g(t.genStrategyTree(1, C))
    ans.append((t.stratName,np.mean(x), max(x)))

In [ ]:
ans

In [ ]:
ans.sort(key = lambda x: list(x)[1:])

In [ ]:
ans

In [ ]:
class Solver(object):
    def __init__(self, words):
        self.words = words
        self.memo = {}
        self.best = {}
    
    def encode(self, sub):
        t = 0
        for w in sub:
            t |= 1 << (len(self.words) - self.words.index(w) - 1)
        return t
    
    def solve(self):
        self.h(self.words)
        return self.memo[self.encode(self.words)]
        
    def h(self, sub, depth = 0):
        c = self.encode(sub)
        if c in self.memo: 
            return self.memo[c]
        
        ans = 1
        if len(sub) == 1:
            self.best[c] = sub[0]
            ans = 1
        elif len(sub) == 2:
            self.best[c] = sub[0]
            ans = 1.5
        else:
            bestScore = -1
            bestGuess = ""
            for g in tqdm(G, disable = depth >= 2):
                t = 1
                splits = getSplits(g, sub, useWords = True)
                if len(splits) == 1: continue
                for res, split in splits.items():
                    if res == '22222': continue
                    t += len(split)/len(sub) * (self.h(split, depth + 1))
#                     if first:
#                         print(split, t, self.h(split))
                
                if bestScore == -1 or t < bestScore:
                    bestScore = t
                    bestGuess = g
                    
#                 print(g,'final ',t, bestScore, getSplits(g, sub, useWords = True))
                
            ans = bestScore
            self.best[c] = bestGuess
        self.memo[c] = ans 
        return ans
    
    def genTree(self, sub = None):
        if sub is None:
            sub = self.words
#         print(self.memo[self.encode(sub)])
        guess = self.best[(self.encode(sub))]
        tree = {
            'guess': guess
        }
        if len(sub) != 1:
            tree['splits'] = {}
            splits = getSplits(guess, sub, useWords = True)
            for res, split in splits.items():
                if res == '22222': continue
                tree['splits'][res] = self.genTree(split)
        
        return tree
        

In [ ]:
s = Solver(words = C)
# s = Solver(words = ['boxer', 'homer', 'hover', 'joker', 'mover', 'roger', 'rover'])
# s = Solver(words = ['other','blood','tares'])
# bin(s.encode(s.words))
s.solve()

In [ ]:
for k,v in s.best.items():
    print(bin(k),'-->',v, s.memo[k])

In [ ]:
for k,v in getSplits('defer',['berry',
 'demur',
 'derby',
 'every',
 'femur',
 'ferry',
 'jerky',
 'leery',
 'lemur',
 'merry',
 'query',
 'reedy'],useWords=True).items():
    print(k,v,bin(s.encode(v)))

In [ ]:
(9*2 + 3*3)/12

In [ ]:
(10*2 + 2*3)/12

In [ ]:
for k,v in getSplits('other',['other','blood','tares'],useWords=True).items():
    print(k,v,bin(s.encode(v)))

In [ ]:
(1 + 7 *2 + 4 * 3)/12

In [ ]:
s.genTree()

In [ ]:
len(s.memo), 2**len(s.words) - 1

In [ ]:
tree

In [9]:
import originalWordLists
from utils import check
import json

In [10]:
with open('originalBestTreeHardMode.json','r') as f:
    tree = json.load(f)

In [11]:
with open('originalBestTreeHardModeList.txt','w') as g:
    for s in originalWordLists.answers:
        ans = []
        curr = tree
        while 1:
            ans.append(curr['guess'])
            if curr['guess'] == s: break
            res = check(curr['guess'], s)
            curr = curr['splits'][res]
        g.write(','.join(ans) + '\n')

In [ ]:
from BruteForceSolver import Solver

In [ ]:
s = Solver(words = C)
s.h(sub=C, depth=2)

In [ ]:
s.genTree(sub=C)

In [2]:
with open('hardModeBaseTruth.txt','r') as f:
    lines = [s.strip().split(',') for s in f]

In [3]:
sum(map(len,lines))

8116

In [4]:
len(lines)

2315

In [7]:
filteredLines = []
for line in lines:
    if line[1] == 'drony':
        filteredLines.append(line)
a,b = len(filteredLines), sum(map(len, filteredLines))
a,b,b/a

(221, 840, 3.8009049773755654)

In [21]:
filteredLines

[['salet', 'drony', 'boofy', 'bobby'],
 ['salet', 'drony', 'boofy', 'comfy'],
 ['salet', 'drony', 'boofy', 'foggy'],
 ['salet', 'drony', 'boofy', 'hobby'],
 ['salet', 'drony', 'boofy', 'poppy'],
 ['salet', 'drony', 'boozy'],
 ['salet', 'drony', 'boozy', 'booby'],
 ['salet', 'drony', 'boozy', 'goofy'],
 ['salet', 'drony', 'boozy', 'woozy'],
 ['salet', 'drony', 'brink'],
 ['salet', 'drony', 'brink', 'bring'],
 ['salet', 'drony', 'brink', 'wring'],
 ['salet', 'drony', 'brink', 'wrung'],
 ['salet', 'drony', 'briny'],
 ['salet', 'drony', 'buddy'],
 ['salet', 'drony', 'buddy', 'biddy'],
 ['salet', 'drony', 'buddy', 'giddy'],
 ['salet', 'drony', 'buddy', 'muddy'],
 ['salet', 'drony', 'buddy', 'pudgy'],
 ['salet', 'drony', 'bunny'],
 ['salet', 'drony', 'bunny', 'funny'],
 ['salet', 'drony', 'bunny', 'ninny'],
 ['salet', 'drony', 'bunny', 'whiny'],
 ['salet', 'drony', 'chirp'],
 ['salet', 'drony', 'chirp', 'birch'],
 ['salet', 'drony', 'chirp', 'quirk'],
 ['salet', 'drony', 'choir'],
 ['salet',

In [16]:
with open('originalBestTreeHardModeList.txt','r') as f:
    lines2 = [s.strip().split(',') for s in f]

In [17]:
c,d = len(lines2), sum(map(len, lines2))
c,d,d/c

(221, 842, 3.8099547511312215)

In [23]:
lines2

[['salet', 'drony', 'biddy'],
 ['salet', 'drony', 'biddy', 'buddy'],
 ['salet', 'drony', 'biddy', 'giddy'],
 ['salet', 'drony', 'biddy', 'muddy'],
 ['salet', 'drony', 'biddy', 'pudgy'],
 ['salet', 'drony', 'birch'],
 ['salet', 'drony', 'birch', 'chirp'],
 ['salet', 'drony', 'birch', 'quirk'],
 ['salet', 'drony', 'boggy', 'bobby'],
 ['salet', 'drony', 'boggy', 'comfy'],
 ['salet', 'drony', 'boggy', 'comfy', 'poppy'],
 ['salet', 'drony', 'boggy', 'foggy'],
 ['salet', 'drony', 'boggy', 'hobby'],
 ['salet', 'drony', 'boozy'],
 ['salet', 'drony', 'boozy', 'booby'],
 ['salet', 'drony', 'boozy', 'goofy'],
 ['salet', 'drony', 'boozy', 'woozy'],
 ['salet', 'drony', 'bring'],
 ['salet', 'drony', 'bring', 'brink'],
 ['salet', 'drony', 'bring', 'wring'],
 ['salet', 'drony', 'bring', 'wrung'],
 ['salet', 'drony', 'briny'],
 ['salet', 'drony', 'brood'],
 ['salet', 'drony', 'brood', 'crowd'],
 ['salet', 'drony', 'brood', 'crowd', 'proud'],
 ['salet', 'drony', 'brown'],
 ['salet', 'drony', 'brown', 'c

In [29]:
x,y = {},{} 
for l,m in zip(filteredLines, lines2):
    x[len(l)] = x.get(len(l),0) + 1
    y[len(m)] = x.get(len(m),0) + 1
x,y

({4: 134, 3: 68, 5: 15, 6: 3, 7: 1}, {3: 69, 4: 133, 5: 14, 6: 4, 7: 2})

In [40]:
sorted([l for l in filteredLines if len(l) > 3 and l[2] == 'nimps'])

[['salet', 'drony', 'nimps', 'cinch'],
 ['salet', 'drony', 'nimps', 'cinch', 'winch'],
 ['salet', 'drony', 'nimps', 'cinch', 'winch', 'finch'],
 ['salet', 'drony', 'nimps', 'cumin'],
 ['salet', 'drony', 'nimps', 'fungi'],
 ['salet', 'drony', 'nimps', 'hunch'],
 ['salet', 'drony', 'nimps', 'hunch', 'bunch'],
 ['salet', 'drony', 'nimps', 'minim'],
 ['salet', 'drony', 'nimps', 'munch'],
 ['salet', 'drony', 'nimps', 'pinch'],
 ['salet', 'drony', 'nimps', 'punch'],
 ['salet', 'drony', 'nimps', 'unzip']]

In [41]:
sorted([l for l in lines2 if len(l) > 3 and l[2] == 'cumin'])

[['salet', 'drony', 'cumin', 'cinch'],
 ['salet', 'drony', 'cumin', 'fungi'],
 ['salet', 'drony', 'cumin', 'minim'],
 ['salet', 'drony', 'cumin', 'munch'],
 ['salet', 'drony', 'cumin', 'pinch'],
 ['salet', 'drony', 'cumin', 'pinch', 'finch'],
 ['salet', 'drony', 'cumin', 'pinch', 'finch', 'winch'],
 ['salet', 'drony', 'cumin', 'punch'],
 ['salet', 'drony', 'cumin', 'punch', 'bunch'],
 ['salet', 'drony', 'cumin', 'punch', 'bunch', 'hunch'],
 ['salet', 'drony', 'cumin', 'unzip']]

In [6]:
# softFilterPossible('salet','00110',G)
from utils import *
softMatch('salet','00110','acyls')

True